In [90]:
import pm4py
from label_functions import *
import editdistance
from functions import *


# import data
def import_xes(file_path):
    event_log = pm4py.read_xes(file_path)
    start_activities = pm4py.get_start_activities(event_log)
    end_activities = pm4py.get_end_activities(event_log)
    print("Start activities: {}\nEnd activities: {}".format(start_activities, end_activities))
    # convert all activity codes to strings
    event_log['Activity code'] = event_log['Activity code'].astype(str)
    return event_log

event_log = import_xes("data/Hospital_log.xes.gz")
print(type(event_log))
print(event_log)

parsing log, completed traces ::   0%|          | 0/1143 [00:00<?, ?it/s]

Start activities: {'1e consult poliklinisch': 71, 'inwend.geneesk.  korte kaart kosten-out': 8, 'verlosk.-gynaec. korte kaart kosten-out': 198, 'vervolgconsult poliklinisch': 234, 'verlosk.-gynaec.   jaarkaart kosten-out': 122, 'cytologisch onderzoek - ectocervix -': 28, 'echografie  - genitalia interna': 44, 'aanname laboratoriumonderzoek': 225, 'ligdagen - alle spec.beh.kinderg.-reval.': 39, 'telefonisch consult': 15, 'behandeltijd - eenheid t3 - megavolt': 1, 'ct abdomen': 2, 'behandeltijd - eenheid t2 - megavolt': 1, 'thorax': 12, 'e.c.g.      - elektrocardiografie': 63, 'cytologisch onderzoek - vagina -': 12, 'mammografie thoraxwand': 3, 'histologisch onderzoek - biopten nno': 19, 'dagverpleging - alle spec.beh.kind.-rev.': 5, 'inwend.geneesk.    jaarkaart kosten-out': 1, 'coupe ter inzage': 19, 'immunopathologisch onderzoek': 10, 'vagina      - scopie incl.evt.vulvabiops': 3, 'punctie tbv cytologisch onderzoek door p': 3, 'cytologisch onderzoek - buiktumorpunctie': 1, 'vrw.gesl.o

In [151]:
def label_function(log_df, required_activities):
    # Filter log to retain only traces with required activities
    filtered_df = pm4py.filtering.filter_event_attribute_values(
        log_df,
        attribute_key='concept:name',
        values=required_activities,
        level='case',
        retain=True,
        case_id_key='case:concept:name'
    )
    
    # Identify case IDs in the filtered DataFrame (these cases are normal)
    normal_cases = set(filtered_df['case:concept:name'].unique())
    
    # Create a dictionary mapping case IDs to 'normal' or 'deviant'
    case_labels = {
        case_id: 'False' if case_id in normal_cases else 'True'
        for case_id in log_df['case:concept:name'].unique()
    }
    
    return case_labels

# Define required activities for this rule
required_activities_f1 = ['ca-19.9 tumormarker', 'ca-125 mbv meia']

# Generate the lf_map using the label function
lf_map = label_function(event_log, required_activities_f1)

## Ignore this markdown comment below.

from functions import *
from sklearn.cluster import DBSCAN
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from Levenshtein import distance as edit_distance
import numpy as np


# Apply label function to the full event log before splitting
lf = label_function_1
# get a label function map
lf_map = get_lf_map(event_log, lf)

# Step 1: Sort traces temporally by the first event's timestamp and split 80/20
event_log_sorted = event_log.sort_values(by='time:timestamp') 
print(event_log_sorted['case:concept:name'].unique())
unique_cases = event_log_sorted['case:concept:name'].unique()

# Split cases into training and testing sets based on 80/20 split
train_cases = unique_cases[:int(0.8 * len(unique_cases))]
test_cases = unique_cases[int(0.8 * len(unique_cases)):]

# Separate event log into training and testing logs
train_log = event_log_sorted[event_log_sorted['case:concept:name'].isin(train_cases)]
test_log = event_log_sorted[event_log_sorted['case:concept:name'].isin(test_cases)]

In [152]:
from functions import *
from sklearn.cluster import DBSCAN
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from Levenshtein import distance as edit_distance
import numpy as np

# Apply label function to the full event log before splitting
#lf = label_function_1
#lf_map = get_lf_map(event_log, lf)

# Step 1: Sort traces temporally by the first event's timestamp and split 80/20
event_log_sorted = event_log.sort_values(by='time:timestamp')
print(event_log_sorted['case:concept:name'])

# Calculate the split index for 80% of the cases, without dropping duplicates
split_index = int(0.8 * len(event_log_sorted))

# Split the sorted event log directly
train_log = event_log_sorted.iloc[:split_index]
test_log = event_log_sorted.iloc[split_index:]

# Verify split sizes
print("Train log size:", len(train_log))
print("Test log size:", len(test_log))


0         00000000
374       00000004
375       00000004
377       00000004
378       00000004
            ...   
150289    00001142
75066     00000591
75065     00000591
60896     00000518
150290    00001142
Name: case:concept:name, Length: 150291, dtype: object
Train log size: 120232
Test log size: 30059


In [153]:

# Step 2: Extract prefixes from training set with specified step sizes
def extract_prefixes(trace, max_len=21, step=5):
    return [trace[:i] for i in range(1, max_len + 1, step) if i <= len(trace)]

# Group and extract prefixes from training traces
train_traces = train_log.groupby('case:concept:name')['Activity code'].apply(list)
all_prefixes = train_traces.apply(lambda x: extract_prefixes(x)).explode()

# Apply label function to each prefix to create lf_map
#lf_map = {i: label_function_1(prefix) for i, prefix in enumerate(all_prefixes)}
#lf_map_train = train_log.groupby('case:concept:name')['Activity code'].apply(label_function_1)
#lf_map_test = test_log.groupby('case:concept:name')['Activity code'].apply(label_function_1)



# Convert prefixes to sequence encoding for DBSCAN
sequence_encoded_prefixes = [' '.join(map(str, prefix)) for prefix in all_prefixes]

In [154]:
print(sequence_encoded_prefixes)

['410100', '410100 419100 10107 410100 419100 339486E', '410100 419100 10107 410100 419100 339486E 390520 390003 390550 379999 370712B', '410100 419100 10107 410100 419100 339486E 390520 390003 390550 379999 370712B 370000 370715A 370000 370403 370407.0', '410100 419100 10107 410100 419100 339486E 390520 390003 390550 379999 370712B 370000 370715A 370000 370403 370407.0 370419 370442 370443 370504A 370000', '419100', '419100 410100 10113 411100 10107 419100', '419100 410100 10113 411100 10107 419100 411100 370437 610001 613000 614400', '419100 410100 10113 411100 10107 419100 411100 370437 610001 613000 614400 710170 370442S 379999 370419S 370000', '419100 410100 10113 411100 10107 419100 411100 370437 610001 613000 614400 710170 370442S 379999 370419S 370000 370000 370000 40014 378808 710170', '10113', '10113 411100 411100 419100 411100 411100', '10113 411100 411100 419100 411100 411100 411100 411100 10213 411100 411100', '10113 411100 411100 419100 411100 411100 411100 411100 10213 4

In [155]:

# Step 3: Compute edit distance matrix for the training set only
n_train = len(sequence_encoded_prefixes)
train_distance_matrix = np.zeros((n_train, n_train))
for i in range(n_train):
    for j in range(i + 1, n_train):
        dist = edit_distance(sequence_encoded_prefixes[i], sequence_encoded_prefixes[j])
        train_distance_matrix[i, j] = dist
        train_distance_matrix[j, i] = dist  # Symmetric matrix

# Step 4: Apply DBSCAN on the training set using the specified parameters
dbscan = DBSCAN(eps=0.125, min_samples=4, metric='precomputed')
#dbscan = DBSCAN(eps=0.15, min_samples=6, metric='precomputed')
#dbscan = DBSCAN(eps=0.2, min_samples=6, metric='precomputed')
train_cluster_labels = dbscan.fit_predict(train_distance_matrix)

In [156]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Define columns for data attributes
X_cols = ['case:Diagnosis code', 'case:Treatment code']

# Initialize label encoders with a placeholder for unknown values
label_encoders = {col: LabelEncoder() for col in X_cols}

# Fit each encoder to the unique values in the corresponding column of the training log
for col in X_cols:
    label_encoders[col].fit(train_log[col].astype(str))  # Ensure data is treated as strings

# Define a function to safely transform data using LabelEncoder, handling unknown values
def safe_transform(encoder, value):
    if value in encoder.classes_:
        return encoder.transform([value])[0]
    else:
        # Adding the unknown label on the fly (or set it to a pre-defined value, like -1)
        return -1  # or handle as desired for unknowns




In [157]:
def encode_data_attributes(event_log, prefixes_with_case_ids, attributes):
    encoded_data = []
    
    # Iterate over each (case_id, prefix)
    for case_id, prefix in prefixes_with_case_ids:
        # Get the last event in the current prefix
        last_event = prefix[-1]
        
        # Filter the event_log for the specific case and last event
        event_rows = event_log[
            (event_log['case:concept:name'] == case_id) & 
            (event_log['Activity code'] == last_event)
        ]
        
        if event_rows.empty:
            print(f"No matching event found for case {case_id} and activity {last_event}")
            # Handle missing events as needed, e.g., skip or fill with default values
            data_vector = [np.nan for _ in attributes]
        else:
            # Assuming events are ordered, take the first occurrence
            event_row = event_rows.iloc[0]
            data_vector = [
                safe_transform(
                    label_encoders[attr], 
                    str(event_row[attr])
                ) 
                for attr in attributes
            ]
        
        # Add the encoded vector to the results
        encoded_data.append(data_vector)
    
    return encoded_data

In [158]:
# Columns used for classifier training
X_cols = ['case:Diagnosis code', 'case:Treatment code']

# Define a threshold for the minimum number of samples required -- OPTIONAL -- TO IMPROVE ROBUSTNESS
min_samples_threshold = 10

# Step 5: Train a classifier for each cluster in the training set
clusters = {}
for cluster_label in np.unique(train_cluster_labels):
    if cluster_label == -1:
        continue  # Skip noise points
    
        # Prepare a list of (case_id, prefix) tuples
    prefixes_with_case_ids = [
        (all_prefixes.index[i], all_prefixes.iloc[i]) 
        for i in cluster_indices
    ]
    
    # Encode data attributes
    X_cluster = encode_data_attributes(train_log, prefixes_with_case_ids, X_cols)
    
    # Get labels for each prefix using lf_map
    y_cluster = [lf_map[case_id] for case_id, _ in prefixes_with_case_ids]
    # Get indices of prefixes in the current cluster
    cluster_indices = [i for i, label in enumerate(train_cluster_labels) if label == cluster_label]
    
     # Print cluster size for inspection
    print(f"Cluster {cluster_label}: Size of X_cluster = {len(X_cluster)}, Size of y_cluster = {len(y_cluster)}")
    

    # Check if the cluster size is too small
    if len(X_cluster) < 5:
        print(f"Warning: Cluster {cluster_label} has very few samples ({len(X_cluster)}). Consider adjusting DBSCAN parameters.")
    
    # Train decision tree classifier for the current cluster
    clf = DecisionTreeClassifier()
    clf.fit(X_cluster, y_cluster)
    clusters[cluster_label] = clf  # Store classifier by cluster label

Cluster 0: Size of X_cluster = 7, Size of y_cluster = 7
Cluster 1: Size of X_cluster = 60, Size of y_cluster = 60
Cluster 2: Size of X_cluster = 195, Size of y_cluster = 195
Cluster 3: Size of X_cluster = 5, Size of y_cluster = 5
Cluster 4: Size of X_cluster = 61, Size of y_cluster = 61
Cluster 5: Size of X_cluster = 14, Size of y_cluster = 14
Cluster 6: Size of X_cluster = 164, Size of y_cluster = 164
Cluster 7: Size of X_cluster = 21, Size of y_cluster = 21
Cluster 8: Size of X_cluster = 120, Size of y_cluster = 120
Cluster 9: Size of X_cluster = 11, Size of y_cluster = 11
Cluster 10: Size of X_cluster = 15, Size of y_cluster = 15
Cluster 11: Size of X_cluster = 9, Size of y_cluster = 9
Cluster 12: Size of X_cluster = 14, Size of y_cluster = 14
Cluster 13: Size of X_cluster = 15, Size of y_cluster = 15
Cluster 14: Size of X_cluster = 12, Size of y_cluster = 12
Cluster 15: Size of X_cluster = 6, Size of y_cluster = 6
Cluster 16: Size of X_cluster = 41, Size of y_cluster = 41
Cluster 1

In [159]:
# Step 6: Define functions for finding the closest cluster and for reliable prediction

def find_closest_cluster(running_trace, train_prefixes, train_cluster_labels):
    # Convert running trace to sequence encoding
    running_sequence = ' '.join(map(str, running_trace))
    # Calculate edit distances between the running trace and each training prefix
    distances = [edit_distance(running_sequence, train_seq) for train_seq in train_prefixes]
    # Get the closest training prefix and its corresponding cluster label
    closest_idx = np.argmin(distances)
    closest_cluster = train_cluster_labels[closest_idx]
    return closest_cluster, closest_idx

def predict_with_reliability(clf, X_test, thresholds=[0.5, 0.6,0.7, 0.8, 0.9], min_class_support=3):
    probs = clf.predict_proba([X_test])[0]
    predicted_class = np.argmax(probs)
    class_support = np.sum(clf.classes_ == predicted_class)
    class_prob = probs[predicted_class]
    if class_support >= min_class_support and any(class_prob >= t for t in thresholds):
        return predicted_class, class_prob
    return None, None

In [160]:
# Inspect keys in lf_map
print("Keys in lf_map:", list(lf_map.items()))  # Display the first few keys for inspection


Keys in lf_map: [('00000000', 'True'), ('00000001', 'True'), ('00000002', 'False'), ('00000003', 'True'), ('00000004', 'False'), ('00000005', 'True'), ('00000006', 'False'), ('00000007', 'False'), ('00000008', 'True'), ('00000009', 'False'), ('00000010', 'False'), ('00000011', 'False'), ('00000012', 'False'), ('00000013', 'True'), ('00000014', 'True'), ('00000015', 'True'), ('00000016', 'True'), ('00000017', 'False'), ('00000018', 'False'), ('00000019', 'True'), ('00000020', 'False'), ('00000021', 'True'), ('00000022', 'True'), ('00000023', 'False'), ('00000024', 'True'), ('00000025', 'True'), ('00000026', 'True'), ('00000027', 'False'), ('00000028', 'True'), ('00000029', 'True'), ('00000030', 'False'), ('00000031', 'True'), ('00000032', 'False'), ('00000033', 'False'), ('00000034', 'True'), ('00000035', 'True'), ('00000036', 'False'), ('00000037', 'True'), ('00000038', 'False'), ('00000039', 'False'), ('00000040', 'True'), ('00000041', 'True'), ('00000042', 'True'), ('00000043', 'True

In [175]:
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Store true and predicted labels
true_labels = []
predicted_labels = []
# Define the sequence of steps
step_sequence = [1, 6, 5, 11, 16, 21]
i = 0
max_prefix_length = 21  # Set the maximum prefix length

# Iterate over each test trace with case IDs
for case_id, test_trace in test_log.groupby('case:concept:name')['Activity code'].apply(list).items():
    reliable_prediction = False
    y_test = None  # Initialize to store the true label for the trace
    y_pred = None  # Initialize to store the predicted label for the trace

    # Access y_test directly using dictionary indexing
    y_test = lf_map.get(case_id, None)
    # Optional: Check if y_test is None and handle the case where the label is not found
    if y_test is None:
        print(f"Warning: No label found for {case_id} in lf_map")

    # Iterate through the test trace based on the custom step sequence
    while i < len(test_trace) and len(test_trace[:i+1]) <= max_prefix_length:
        current_prefix = test_trace[:i+1]  # i+1 to capture up to the current step
        print(len(current_prefix))
        
        # Check if the prefix length exceeds the maximum
        if len(current_prefix) > max_prefix_length:
            break  # Exit the loop if the prefix length exceeds the limit

        # Find the closest cluster using minimum edit distance
        closest_cluster, closest_idx = find_closest_cluster(current_prefix, sequence_encoded_prefixes, train_cluster_labels)

        if closest_cluster == -1:
            i += step_sequence[0] if len(step_sequence) > 0 else 5  # Default step if sequence is empty
            continue  # Skip if the closest cluster is noise

        clf = clusters.get(closest_cluster)
        if clf:
            prefixes_with_case_ids = [(case_id, current_prefix)]
            X_test = encode_data_attributes(test_log, prefixes_with_case_ids, X_cols)[0]
            y_pred, confidence = predict_with_reliability(clf, X_test)

            if y_pred is not None:
                print(f"Reliable prediction achieved: {y_pred} with confidence {confidence}")
                reliable_prediction = True
                break  # Stop if a reliable prediction is made

        # Move to the next step in the sequence
        i += step_sequence.pop(0) if step_sequence else 5  # Default step of 5 if sequence is exhausted
    # Append the final true and predicted labels for the current trace
    true_labels.append(y_test)
    predicted_labels.append(y_pred if reliable_prediction else None)

    if not reliable_prediction:
        print("Prediction failure: No reliable prediction achieved by end of trace.")


1
Prediction failure: No reliable prediction achieved by end of trace.
2
Prediction failure: No reliable prediction achieved by end of trace.
Prediction failure: No reliable prediction achieved by end of trace.
8
13
18
Prediction failure: No reliable prediction achieved by end of trace.
Prediction failure: No reliable prediction achieved by end of trace.
Prediction failure: No reliable prediction achieved by end of trace.
Prediction failure: No reliable prediction achieved by end of trace.
Prediction failure: No reliable prediction achieved by end of trace.
Prediction failure: No reliable prediction achieved by end of trace.
Prediction failure: No reliable prediction achieved by end of trace.
Prediction failure: No reliable prediction achieved by end of trace.
Prediction failure: No reliable prediction achieved by end of trace.
Prediction failure: No reliable prediction achieved by end of trace.
Prediction failure: No reliable prediction achieved by end of trace.
Prediction failure: No

In [176]:

# Calculate F1 Score after all predictions
f1 = f1_score(true_labels, predicted_labels, average='binary')  # Use 'binary' if it's a binary classification problem
print(f'F1 Score: {f1:.3f}')

ValueError: Classification metrics can't handle a mix of binary and unknown targets

In [101]:

# Calculate and display confusion matrix
cm = confusion_matrix(true_labels, predicted_labels, labels=np.unique(true_labels + predicted_labels))
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(true_labels + predicted_labels),
            yticklabels=np.unique(true_labels + predicted_labels))
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()


ValueError: 'labels' should contains at least one label.

## Ignore this markdown comment below.

import numpy as np
from sklearn.metrics import f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Store true and predicted labels
true_labels = []
predicted_labels = []

# Step 7: Replay each test trace for prediction
for test_trace in test_log.groupby('case:concept:name')['Activity code'].apply(list):
    reliable_prediction = False
    y_test = None  # Initialize to store the true label for the trace
    y_pred = None  # Initialize to store the predicted label for the trace

    # Try to retrieve the true label for the trace
    try:
        y_test = lf_map.loc[test_trace[0]]  # Access y_test using the label map
    except KeyError:
        print(f"Warning: No label found for {test_trace[0]} in lf_map")
        continue  # Skip processing if the label is not found

    # Incrementally generate prefixes every 5 events (replay logic)
    for i in range(1, len(test_trace) + 1, 5):  # Replay every 5 events
        current_prefix = test_trace[:i]
        
        # Find the closest cluster using minimum edit distance
        closest_cluster, closest_idx = find_closest_cluster(current_prefix, sequence_encoded_prefixes, train_cluster_labels)
        
        if closest_cluster == -1:
            continue  # Skip if the closest cluster is noise

        clf = clusters.get(closest_cluster)
        if clf:
            X_test = encode_data_attributes(test_log, [current_prefix], X_cols)[0]
            y_pred, confidence = predict_with_reliability(clf, X_test)
            
            if y_pred is not None:
                print(f"Reliable prediction achieved: {y_pred} with confidence {confidence}")
                reliable_prediction = True
                break  # Stop if a reliable prediction is made

    # Append the final true and predicted labels for the current trace
    true_labels.append(y_test)
    predicted_labels.append(y_pred if reliable_prediction else None)

    if not reliable_prediction:
        print("Prediction failure: No reliable prediction achieved by end of trace.")
